In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression,f_classif
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn import svm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
from math import sqrt
 
import torch
import torch.nn as nn
 
class SelfAttention(nn.Module):
    def __init__(self, dim_q, dim_k, dim_v):
        super(SelfAttention, self).__init__()
        self.dim_q = dim_q
        self.dim_k = dim_k
        self.dim_v = dim_v
 
        self.linear_q = nn.Linear(dim_q, dim_k,bias=False)
        self.linear_k = nn.Linear(dim_q, dim_k,bias=False)
        self.linear_v = nn.Linear(dim_q, dim_v,bias=False)
        self._norm_fact = 1 / sqrt(dim_k)
 
    def forward(self, x):
 
        batch, dim_q = x.shape
        n = 1
        assert dim_q == self.dim_q
        q = self.linear_q(x)
        k = self.linear_k(x)
        v = self.linear_v(x)

        dist = torch.mm(q, k.transpose(0,1)) * self._norm_fact
        dist = torch.softmax(dist, dim=-1) 
        att = torch.mm(dist, v)
        return att

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
import torch
from torch import tensor
torch.set_default_tensor_type(torch.DoubleTensor)
from shutil import copyfile
Best_ACC=0
Best_RE=0
Best_F1=0
Best_FLAG=0
LOW_ACC=1.1
LOW_RE=1.1
LOW_F1=1.1
LOW_FLAG=0
AVG_ACC=0
DF=0;
DB=0;
DV=0;
DM=0;
op_now=0
ACCNOW=[0 for i in range(825)]
POS_record = [[0 for i in range(825)] for j in range(161)]
NEV_record = [[0 for j in range(825)] for i in range(161)]
X_bian_P = np.array([[0 for j in range(161)] for i in range(825)],dtype='float')
X_bian_N = np.array([[0 for j in range(161)] for i in range(825)],dtype='float')
POS = [0 for i in range(161)]
NEV = [0 for i in range(161)]
fl=12

for iii in range(50,250,1):
    data=scio.loadmat('./floyd'+str(100-fl)+'_'+str(fl)+'.mat')
    X ={ }
    data_C ={ }
    data_W ={ }
    XXX_2d = [[0 for i in range(13858)]for j in range(161)]
    for i in range(161):
        X[i]=data['Z'][i]
#     print(X[0][0])
    t=0
    for i in range(1,162):
        data_C[i]=scio.loadmat('./floyd'+str(100-fl)+'_'+str(fl)+'_'+str(i)+'.mat')
    data_W=scio.loadmat('./floyd'+str(100-fl)+'_'+str(fl)+'.mat')
    data_CC=np.array([[0 for i in range(164)]for j in range(161)],dtype='float')
    data_LL=np.array([0 for i in range(161)],dtype='float')
    data_WW=np.array([0 for i in range(161)],dtype='float')
    for i in range(1,162):
        for j in range(164):
            data_CC[i-1][j]=np.array(data_C[i]['C'][j],dtype='float')
    for i in range(161):
        data_WW[i]=np.array(data_W['Smallworld'][0][i],dtype='float')
    for i in range(161):
        t=0
        for k in range(0,164):
            for l in range(k+1,164):
                XXX_2d[i][t]=np.array(X[i][k,l])
                t=t+1
        for k in range(0,164):
            for l in range(164,167):
                if (l==164):
                    XXX_2d[i][t]=np.array(X[i][k,l])*data_CC[i][k]*data_WW[i]
                else:
                    XXX_2d[i][t]=np.array(X[i][k,l])
                t=t+1
    XX=np.array(XXX_2d,dtype='float')
    selector = SelectKBest(f_regression, k=iii)
    X_train =np.array([[0 for i in range(13858)] for j in range(138)],dtype='float')
    X_test =np.array([[0 for i in range(13858)] for j in range(23)],dtype='float')
    Y_train=np.array([0 for i in range(138)],dtype='int')
    Y_test=np.array([0 for i in range(23)],dtype='int')
    X_trainA =np.array([[0 for i in range(iii+7)] for j in range(138)],dtype='float')
    X_testA =np.array([[0 for i in range(iii+7)] for j in range(23)],dtype='float')
    Y_trainA=np.array([0 for i in range(138)],dtype='int')
    Y_testA=np.array([0 for i in range(23)],dtype='int')
    
    POS_record_train = [[0 for i in range(825)] for j in range(138)]
    NEV_record_train = [[0 for j in range(825)] for i in range(138)]
    X_bian_P_train = np.array([[0 for j in range(138)] for i in range(825)],dtype='float')
    X_bian_N_train = np.array([[0 for j in range(138)] for i in range(825)],dtype='float')
    POS_train = [0 for i in range(138)]
    NEV_train = [0 for i in range(138)]
    
    POS_record_test = [[0 for i in range(825)] for j in range(23)]
    NEV_record_test = [[0 for j in range(825)] for i in range(23)]
    X_bian_P_test = np.array([[0 for j in range(23)] for i in range(825)],dtype='float')
    X_bian_N_test = np.array([[0 for j in range(23)] for i in range(825)],dtype='float')
    POS_test = [0 for i in range(23)]
    NEV_test = [0 for i in range(23)]
    
    u,v=0,0;
    for j in range(161):
        if (j%7==op_now):
            X_test[u]=XX[j]
            Y_test[u]=YY[j]
            u=u+1;
        else:
            X_train[v]=XX[j]
            Y_train[v]=YY[j]
            v=v+1;
    X_newR = selector.fit_transform(X_train,Y_train)
    X_newE = selector.transform(X_test)
    
    for jb in range(0,iii):
        for ja in range(138):
            if (X_newR[ja][jb]>0):
                POS_record_train[ja][POS_train[ja]]=jb
                X_bian_P_train[POS_train[ja]][ja]=np.array(X_newR[ja][jb],dtype='float')
                POS_train[ja]=POS_train[ja]+1
            if (X_newR[ja][jb]<0):
                NEV_record_train[ja][NEV_train[ja]]=jb
                X_bian_N_train[NEV_train[ja]][ja]=np.array(X_newR[ja][jb],dtype='float')
                NEV_train[ja]=NEV_train[ja]+1
    
    for jb in range(0,iii):
        for ja in range(23):
            if (X_newE[ja][jb]>0):
                POS_record_test[ja][POS_test[ja]]=jb
                X_bian_P_test[POS_test[ja]][ja]=np.array(X_newE[ja][jb],dtype='float')
                POS_test[ja]=POS_test[ja]+1
            if (X_newE[ja][jb]<0):
                NEV_record_test[ja][NEV_test[ja]]=jb
                X_bian_N_test[NEV_test[ja]][ja]=np.array(X_newE[ja][jb],dtype='float')
                NEV_test[ja]=NEV_test[ja]+1
    
    for kkk in range(50):
        X_newRA = np.array([[0 for i in range(iii+7)] for j in range(138)],dtype='float')
        X_newEA = np.array([[0 for i in range(iii+7)] for j in range(23)],dtype='float')
        Feature=np.zeros((iii+7,161))
        u,v=0,0;
        
        for i in range(161):
            if (i%7==op_now):
                for j in range(iii):
                    X_newEA[u][j]=X_newE[u][j]
                t=iii
                for j in range(7):
                    X_newEA[u][t]=np.array(X_2d[i][j],dtype=float)
                    t=t+1
                u=u+1
            else:
                for j in range(iii):
                    X_newRA[v][j]=X_newR[v][j]
                t=iii
                for j in range(7):
                    X_newRA[v][t]=np.array(X_2d[i][j],dtype=float)
                    t=t+1
                v=v+1
        
        u,v=0,0
        for lll in range(161):
            if (lll%7==op_now):
                tagAA1=torch.zeros(POS_test[u])
                tagAA2=torch.zeros(NEV_test[u])
                if (POS_test[u]!=0):
                    model1=SelfAttention(1,POS_test[u],POS_test[u])
                    Feature1=np.zeros((POS_test[u],1))
                    for j in range(POS_test[u]):
                         Feature1[j][0]=np.array(X_bian_P_test[j][u])
                    Feature_tensor = torch.from_numpy(Feature1)
                    YQ1=model1(Feature_tensor)
                    tagAA1=torch.sum(YQ1,dim=1)
                    for j in range(POS_test[u]): 
                        X_newEA[u][POS_record_test[u][j]]=X_newEA[u][POS_record_test[u][j]]*(tagAA1[j].cpu().detach().numpy())
                    PATH = "./SA_"+str(lll)+"_"+str(op_now)+"A.pth"
                    torch.save(model1.state_dict(), PATH)
                if (NEV_test[u]!=0):
                    model2=SelfAttention(1,NEV_test[u],NEV_test[u])
                    Feature2=np.zeros((NEV_test[u],1))
                    for j in range(NEV_test[u]):
                         Feature2[j][0]=np.array(X_bian_N_test[j][u])
                    Feature_tensor = torch.from_numpy(Feature2)
                    YQ2=model2(Feature_tensor)
                    tagAA2=torch.sum(YQ2,dim=1)
                    PATH = "./SA_"+str(lll)+"_"+str(op_now)+"B.pth"
                    torch.save(model2.state_dict(), PATH)
                    for j in range(NEV_test[u]):
                        X_newEA[u][NEV_record_test[u][j]]=X_newEA[u][NEV_record_test[u][j]]*(tagAA2[j].cpu().detach().numpy())
                u=u+1
            else:
                tagAA1=torch.zeros(POS_train[v])
                tagAA2=torch.zeros(NEV_train[v])
                if (POS_train[v]!=0):
                    model1=SelfAttention(1,POS_train[v],POS_train[v])
                    Feature1=np.zeros((POS_train[v],1))
                    for j in range(POS_train[v]):
                         Feature1[j][0]=np.array(X_bian_P_train[j][v])
                    Feature_tensor = torch.from_numpy(Feature1)
                    YQ1=model1(Feature_tensor)
                    tagAA1=torch.sum(YQ1,dim=1)
                    for j in range(POS_train[v]): 
                        X_newRA[v][POS_record_train[v][j]]=X_newRA[v][POS_record_train[v][j]]*(tagAA1[j].cpu().detach().numpy())
                    PATH = "./SA_"+str(lll)+"_"+str(op_now)+"A.pth"
                    torch.save(model1.state_dict(), PATH)
                if (NEV_train[v]!=0):
                    model2=SelfAttention(1,NEV_train[v],NEV_train[v])
                    Feature2=np.zeros((NEV_train[v],1))
                    for j in range(NEV_train[v]):
                         Feature2[j][0]=np.array(X_bian_N_train[j][v])
                    Feature_tensor = torch.from_numpy(Feature2)
                    YQ2=model2(Feature_tensor)
                    tagAA2=torch.sum(YQ2,dim=1)
                    PATH = "./SA_"+str(lll)+"_"+str(op_now)+"B.pth"
                    torch.save(model2.state_dict(), PATH)
                    for j in range(NEV_train[v]):
                        X_newRA[v][NEV_record_train[v][j]]=X_newRA[v][NEV_record_train[v][j]]*(tagAA2[j].cpu().detach().numpy())
                v=v+1
        
        X_newEA = scaler.fit_transform(X_newEA)
        X_newRA = scaler.fit_transform(X_newRA)

        u,v=0,0;
        for i in range(161):
            if (i%7==op_now):
                for j in range(iii+7):
                    X_testA[u][j]=X_newEA[u][j]
                u=u+1
            else:
                for j in range(iii+7):
                    X_trainA[v][j]=X_newRA[v][j]
                v=v+1
        ACC=0;
        cnt=0;
        clf_linear = svm.SVC(kernel='linear',C=0.095)
        clf_linear.fit(X_trainA,Y_train)
        score_linear_test = clf_linear.score(X_testA,Y_test)
        score_linear_train = clf_linear.score(X_trainA,Y_train)
        predict_test = clf_linear.predict(X_testA)
#         print("SVM Test  Accuracy : %.4g" % (score_linear_test))
#         print("SVM Train  Accuracy : %.4g" % (score_linear_train))
        ACC=ACC+score_linear_test;
        cnt=cnt+1;
        ACC=ACC/cnt
        AVG_ACC=AVG_ACC+ACC
        if ACC>Best_ACC:
            Best_FLAG=iii
            Best_ACC=ACC
        if ACC<LOW_ACC:
            LOW_FLAG=iii
            LOW_ACC=ACC
        print(iii,kkk,ACC)
        if (ACCNOW[iii]<ACC):
            ACCNOW[iii]=ACC
            for lll in range(161):
                copyfile("./SA_"+str(lll)+"_"+str(op_now)+"A.pth", "./SA_FINAL_"+str(iii)+"_"+str(lll)+"_"+str(op_now)+"A.pth")
                copyfile("./SA_"+str(lll)+"_"+str(op_now)+"B.pth", "./SA_FINAL_"+str(iii)+"_"+str(lll)+"_"+str(op_now)+"B.pth")
    print(iii,ACCNOW[iii])
print("BEST:")
print(Best_FLAG,Best_ACC)
print(LOW_FLAG,LOW_ACC)
print("ACC")